In [ ]:
import pandas as pd

import glob

from google.oauth2 import service_account
# from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
voter_history_directory = r'D:\Users\bengen343\Documents\Master Voting History'

# Google Service Credential JSON <- You'll need your own!
# gs_credentials = ServiceAccountCredentials.from_json_keyfile_name('cpc-datawarehouse-51210-cbbf1375a84e.json')
# Establish BigQuery credentials
bq_credentials = service_account.Credentials.from_service_account_file(
    'cpc-datawarehouse-51210-cbbf1375a84e.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bq_project_id = 'cpc-datawarehouse-51210'
bq_table_id = 'cpc-datawarehouse-51210.co_voterfile.vote_history'

bq_schema_list = [
    {'name': 'VOTER_ID', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'ELECTION_TYPE', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'ELECTION_DATE', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'ELECTION_DESCRIPTION', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'VOTING_METHOD', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'PARTY', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'COUNTY_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}
]

In [ ]:
# Load vote history data frame
print("Beginning to assemble vote history file.")

allFiles = glob.glob(voter_history_directory + "/*.txt")
history_df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, sep=",", encoding='cp437', index_col=None, header=0, low_memory=False)
    list_.append(df)
history_df = pd.concat(list_)

history_df.reset_index(drop=True, inplace=True)

In [ ]:
history_df.to_gbq(destination_table=bq_table_id, project_id=bq_project_id, if_exists='replace', table_schema=bq_schema_list, credentials=bq_credentials)